In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
np.random.seed(0)
import nltk
from nltk import FreqDist, word_tokenize
from nltk.corpus import stopwords
from nltk.collocations import *
import string
import re

In [2]:
df = pd.read_csv('data/judge-1377884607_tweet_product_company.csv', encoding='latin1')
df.head(50)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product
6,NaN,NaN,No emotion toward brand or product
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion


In [3]:
df_1 = df.copy()

In [4]:
df_1.rename(columns={'emotion_in_tweet_is_directed_at': 'product/brand','is_there_an_emotion_directed_at_a_brand_or_product':'target'}, 
          inplace=True)
df_1.head()

,tweet_text,product/brand,target
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [7]:
df_1 = df_1.replace({'No emotion toward brand or product':'neutral'})
df_1 = df_1.replace({"I can't tell":'neutral'})
df_1 = df_1.replace({'Positive emotion':'positive'})
df_1 = df_1.replace({'Negative emotion':'negative'})

In [8]:
df_1['target'].value_counts()

neutral     5545
positive    2978
negative     570
Name: target, dtype: int64

In [9]:
sub = 'neutral'
df1 = df_1[df_1['target'].str.contains(sub)]
df1

,tweet_text,product/brand,target
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,neutral
6,NaN,NaN,neutral
16,Holler Gram for iPad on the iTunes App Store -...,NaN,neutral
32,"Attn: All #SXSW frineds, @mention Register fo...",NaN,neutral
33,Anyone at #sxsw want to sell their old iPad?,NaN,neutral
...,...,...,...
9087,"@mention Yup, but I don't have a third app yet...",NaN,neutral
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,neutral
9090,"Google's Zeiger, a physician never reported po...",NaN,neutral
9091,Some Verizon iPhone customers complained their...,NaN,neutral


In [ ]:
text = df1['tweet_text'].str.cat(sep=', ')
brands = re.findall("(Apple)|(Google)|(iPad)|(iPad App)|(Android)", text)
brands

In [19]:
df1['tweet_text'][32:33]

78    iPad2? RT @mention Droid &amp; Mac here :) RT ...
Name: tweet_text, dtype: object

In [20]:
sub1 = 'Apple'
sub2 = 'Google'
sub3 = 'iPad'
sub4 = 'App'
sub5 = 'Android'
all_sub = sub1 + sub2 + sub3 + sub4 + sub5

In [22]:
df1 = df1[df1['tweet_text'].notna()]

In [23]:
df1

,tweet_text,product/brand,target
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,neutral
16,Holler Gram for iPad on the iTunes App Store -...,NaN,neutral
32,"Attn: All #SXSW frineds, @mention Register fo...",NaN,neutral
33,Anyone at #sxsw want to sell their old iPad?,NaN,neutral
34,Anyone at #SXSW who bought the new iPad want ...,NaN,neutral
...,...,...,...
9087,"@mention Yup, but I don't have a third app yet...",NaN,neutral
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,neutral
9090,"Google's Zeiger, a physician never reported po...",NaN,neutral
9091,Some Verizon iPhone customers complained their...,NaN,neutral


In [21]:
df2 = df1[df1['tweet_text'].str.contains(all_sub)]
df1

ValueError: Cannot mask with non-boolean array containing NA / NaN values